In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from bs4 import BeautifulSoup
import re
import time

In [2]:
driver = Chrome()

In [3]:
driver.get("https://portal.korea.ac.kr/front/Intro.kpd")

In [4]:
with open('idpw.txt','r') as f:
    data=f.read()

In [5]:
id_,pw_=re.split(r'(\n)',data)[0],re.split(r'(\n)',data)[2]

In [6]:
driver.find_element_by_name('id').clear()
driver.find_element_by_name('id').send_keys(id_)

In [7]:
driver.find_element_by_name('pw').clear()
driver.find_element_by_name('pw').send_keys(pw_)

In [8]:
time.sleep(1)
driver.find_element_by_name('loginsubmit').click()

In [13]:
#포탈 첫화면에서 등록/장학 클릭
driver.find_element_by_xpath('//*[@id="header"]/div[2]/div/div/ul/li[2]/a').click()
time.sleep(2)

In [86]:
def surfer():
    page_num=1
    while True:
        if page_num==1:
            driver.switch_to.frame("_component")
        tab_num=len(driver.find_elements_by_xpath('//*[@id="Search"]/table/tbody/tr'))#테이블 아이탬 몇개있는지 확인
        print(f"페이지 번호 : {page_num}, 탭 수: {tab_num}")
        for i in range(1,tab_num+1):
            if i!=1:#볼드체 제목 클릭
                driver.switch_to.frame("_component")
            path=('//*[@id="Search"]/table/tbody/tr[%d]/td[3]/b/a' %i)
            try: #일반 제목 클릭
                element=driver.find_element_by_xpath(path)
            except:
                path=('//*[@id="Search"]/table/tbody/tr[%d]/td[3]/a' %i)
                element=driver.find_element_by_xpath(path)

            element.click()#여기가 클릭하는 부분 (이밑으로 크롤러 넣으면 됨)
            print(path)
            time.sleep(1)
            driver.back()
        driver.switch_to.frame("_component")
        btn_next=driver.find_element_by_css_selector('#Search > div.paging > div > a.btn.next')
        #장학금 공지는 div[2]/div/a[5], 지난 장학금 공지는 /div[3]/div/a[7]
        if btn_next.get_attribute('href')[-4:]=='prev':#맨마지막 페이지 도달 하면 break
            print("끝도달")
            break
        btn_next.click()
        page_num+=1
        time.sleep(1)

In [91]:
driver.find_element_by_xpath('//*[@id="m102"]/a').click() #장학 클릭

In [92]:
driver.find_element_by_xpath('//*[@id="sm110"]/a').click() #지난 장학금 공지 클릭

In [93]:
driver.switch_to.frame("_component")

In [94]:
year=list(range(2010,2022))

In [95]:
driver.find_element_by_xpath('//*[@id="Search"]/div[2]/div[1]/div/input[1]').clear()#년도창비우기
driver.find_element_by_xpath('//*[@id="Search"]/div[2]/div[1]/div/input[1]').send_keys(year[3])#년도 입력
driver.find_element_by_xpath('//*[@id="Search"]/div[2]/div[1]/span[2]/input').click()#검색 클릭
driver.find_element_by_css_selector('#Search > div.paging > div > a.btn.first').click()#1페이지로 초기화

In [96]:
driver.back()

In [97]:
surfer()

페이지 번호 : 1, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 2, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Search"]/table/tbody/tr[2]/td[3]/a
//*[@id="Search"]/table/tbody/tr[3]/td[3]/a
//*[@id="Search"]/table/tbody/tr[4]/td[3]/a
//*[@id="Search"]/table/tbody/tr[5]/td[3]/a
//*[@id="Search"]/table/tbody/tr[6]/td[3]/a
//*[@id="Search"]/table/tbody/tr[7]/td[3]/a
//*[@id="Search"]/table/tbody/tr[8]/td[3]/a
//*[@id="Search"]/table/tbody/tr[9]/td[3]/a
//*[@id="Search"]/table/tbody/tr[10]/td[3]/a
페이지 번호 : 3, 탭 수: 10
//*[@id="Search"]/table/tbody/tr[1]/td[3]/a
//*[@id="Searc